# Hello world
and with this humble words, this masterpiece begin

In [ ]:
# Let's load the data into memory
!python3 load_data.py

In [2]:
# Libraries
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns

# Scikit-learn
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


# TensorFlow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import pickle  # Para guardar el pipeline
import joblib  # Para guardar el pipeline

# Import the KaggleHub library
import kagglehub
from kagglehub import KaggleDatasetAdapter


2025-01-24 22:17:20.432249: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-24 22:17:20.434857: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-24 22:17:20.441468: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737775040.451911  156636 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737775040.455039  156636 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-24 22:17:20.467497: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [3]:
# Import the necessary functions from utilities.py
from utilities import get_preprocessor, get_pipeline

In [4]:
# Load a DataFrame with a specific version of a CSV
df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "ranadeep/credit-risk-dataset/versions/3",
    "loan/loan.csv",
)

/home/s/code/unal/RNA/Trabajos/modelos-riesgo-credito/.venv/lib/python3.11/site-packages/kagglehub/pandas_datasets.py:92: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  result = read_function(


In [5]:
pipeline = get_pipeline()

# Preprocess the data
preproccesed_df = pipeline.fit_transform(df)

In [6]:
# Dividir los datos en variables predictoras (X) y la variable objetivo (y)
X = preproccesed_df.drop(columns=['funded_amnt', 'acc_now_delinq', 'total_rec_int', 'loan_status','collection_recovery_fee',
       'revol_util', 'out_prncp', 'policy_code', 'collections_12_mths_ex_med',
       'recoveries', 'total_rev_hi_lim', 'tot_cur_bal', 'total_rec_late_fee',
       'total_acc', 'total_rec_prncp'])
y = preproccesed_df['loan_status'].astype(int)

In [7]:
numerical_columns = X.select_dtypes(include=['float64', 'int64']).columns
categorical_columns = X.select_dtypes(include=['object']).columns
preprocessor = get_preprocessor(X)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2025)

In [9]:
# Aplicar el preprocesamiento a los datos
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

In [10]:
# Construir el modelo de Keras
model = Sequential([
    Dense(64, input_dim=X_train_preprocessed.shape[1], activation='relu'),  # Capa de entrada con 64 neuronas
    Dense(32, activation='relu'),  # Capa oculta con 32 neuronas
    Dense(1, activation='sigmoid')  # Capa de salida (sigmoide para clasificación binaria)
])

/home/s/code/unal/RNA/Trabajos/modelos-riesgo-credito/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-01-24 22:17:44.946830: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [11]:
# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'recall'])

In [12]:
# Entrenar el modelo
model.fit(X_train_preprocessed, y_train, epochs=10, batch_size=32, verbose=1)

Epoch 1/10
22185/22185 ━━━━━━━━━━━━━━━━━━━━ 13s 555us/step - accuracy: 0.9665 - loss: 0.1294 - recall: 0.9990
Epoch 2/10
22185/22185 ━━━━━━━━━━━━━━━━━━━━ 12s 556us/step - accuracy: 0.9724 - loss: 0.1111 - recall: 0.9993
Epoch 3/10
22185/22185 ━━━━━━━━━━━━━━━━━━━━ 12s 555us/step - accuracy: 0.9728 - loss: 0.1090 - recall: 0.9994
Epoch 4/10
22185/22185 ━━━━━━━━━━━━━━━━━━━━ 12s 550us/step - accuracy: 0.9730 - loss: 0.1078 - recall: 0.9994
Epoch 5/10
22185/22185 ━━━━━━━━━━━━━━━━━━━━ 12s 551us/step - accuracy: 0.9732 - loss: 0.1072 - recall: 0.9994
Epoch 6/10
22185/22185 ━━━━━━━━━━━━━━━━━━━━ 12s 548us/step - accuracy: 0.9732 - loss: 0.1069 - recall: 0.9994
Epoch 7/10
22185/22185 ━━━━━━━━━━━━━━━━━━━━ 12s 550us/step - accuracy: 0.9732 - loss: 0.1067 - recall: 0.9994
Epoch 8/10
22185/22185 ━━━━━━━━━━━━━━━━━━━━ 12s 547us/step - accuracy: 0.9734 - loss: 0.1064 - recall: 0.9995
Epoch 9/10
22185/22185 ━━━━━━━━━━━━━━━━━━━━ 12s 547us/step - accuracy: 0.9736 - loss: 0.1056 - recall: 0.9996
Epoch 10/1

In [13]:
# Hacer predicciones
y_pred = (model.predict(X_test_preprocessed) > 0.5).astype("int32")
# Mostrar el reporte de clasificación
print(classification_report(y_test, y_pred))

5547/5547 ━━━━━━━━━━━━━━━━━━━━ 2s 289us/step
              precision    recall  f1-score   support

           0       0.98      0.66      0.79     13324
           1       0.97      1.00      0.99    164152

    accuracy                           0.97    177476
   macro avg       0.98      0.83      0.89    177476
weighted avg       0.97      0.97      0.97    177476



In [14]:
# Save the preprocessor
filename = 'preprocessor.pkl'
with open(filename, 'wb') as f:
    pickle.dump(preprocessor, f)

# Save the model to disk with pickle
filename = 'model.pkl'
joblib.dump(model, filename)

['model.pkl']

# Explore a little bit

In [15]:
column_data = X_test.iloc[0]

In [16]:
column_data

loan_amnt                           24000.0
funded_amnt_inv                     24000.0
term                                    0.0
int_rate                               6.03
installment                          730.46
sub_grade                               7.8
home_ownership                     MORTGAGE
annual_inc                         182000.0
verification_status                     1.0
pymnt_plan                                n
dti                                   13.15
delinq_2yrs                             0.0
inq_last_6mths                          1.0
mths_since_last_delinq                 72.0
mths_since_last_record                 70.0
open_acc                               15.0
pub_rec                                 0.0
revol_bal                           26660.0
initial_list_status                       f
out_prncp_inv                           0.0
total_pymnt                    25479.895843
total_pymnt_inv                     25479.9
last_pymnt_amnt                 

In [17]:
# make a prediction with the model just a test row
input_data = preprocessor.transform(column_data.to_frame().T)
model.predict(input_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


array([[0.99998355]], dtype=float32)